# Student Performance Prediction using Artificial Neural Networks

This notebook demonstrates how to predict student academic performance using Artificial Neural Networks (ANN). We'll use a dataset containing various student attributes and their corresponding academic scores to build a predictive model.

In [ ]:
# Import necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import matplotlib.pyplot as plt # for visualization
import seaborn as sns # for statistical data visualization

# For model building and evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# For deep learning
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

## Data Loading and Exploration

For this project, we'll use the Student Performance Dataset which contains information about students' demographics, social factors, and academic performance.

In [ ]:
# Load the dataset
# Note: In a real environment, you would replace this with your actual data path
# For demonstration, we'll create a synthetic dataset

# Create a synthetic student performance dataset
np.random.seed(42)
n_samples = 500

# Generate features
age = np.random.normal(18, 2, n_samples).round(1)
study_time = np.random.normal(6, 2, n_samples).round(1)
absences = np.random.poisson(5, n_samples)
previous_grades = np.random.normal(70, 15, n_samples).round(1)
parent_education = np.random.randint(0, 5, n_samples)
internet_access = np.random.randint(0, 2, n_samples)
extra_activities = np.random.randint(0, 2, n_samples)

# Generate target variable (final grade) with some correlation to features
final_grade = (0.1 * age + 0.3 * study_time - 0.2 * absences + 
               0.4 * previous_grades + 0.1 * parent_education + 
               0.05 * internet_access + 0.05 * extra_activities)

# Add some noise
final_grade = final_grade + np.random.normal(0, 5, n_samples)

# Ensure grades are within reasonable bounds (0-100)
final_grade = np.clip(final_grade, 0, 100).round(1)

# Create DataFrame
data = {
    'Age': age,
    'StudyTime': study_time,
    'Absences': absences,
    'PreviousGrades': previous_grades,
    'ParentEducation': parent_education,
    'InternetAccess': internet_access,
    'ExtraActivities': extra_activities,
    'FinalGrade': final_grade
}

df = pd.DataFrame(data)

# Display the first few rows
df.head()

In [ ]:
# Check the shape of the dataset
df.shape

In [ ]:
# Get information about the dataset
df.info()

In [ ]:
# Check for missing values
df.isnull().sum()

In [ ]:
# Statistical summary of the dataset
df.describe()

In [ ]:
# Check for duplicates
df.duplicated().sum()

## Data Visualization

In [ ]:
# Correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Distribution of final grades
plt.figure(figsize=(10, 6))
sns.histplot(df['FinalGrade'], kde=True)
plt.title('Distribution of Final Grades')
plt.xlabel('Final Grade')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Relationship between study time and final grade
plt.figure(figsize=(10, 6))
sns.scatterplot(x='StudyTime', y='FinalGrade', data=df)
plt.title('Study Time vs Final Grade')
plt.xlabel('Study Time (hours/week)')
plt.ylabel('Final Grade')
plt.show()

In [ ]:
# Relationship between previous grades and final grade
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PreviousGrades', y='FinalGrade', data=df)
plt.title('Previous Grades vs Final Grade')
plt.xlabel('Previous Grades')
plt.ylabel('Final Grade')
plt.show()

## Data Preprocessing

In [ ]:
# Separate features and target variable
X = df.drop('FinalGrade', axis=1)
y = df['FinalGrade']

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Building the Neural Network Model

In [ ]:
# Define the model architecture
model = Sequential()

# Input layer and first hidden layer
model.add(Dense(16, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dropout(0.2))  # Add dropout for regularization

# Second hidden layer
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(1, activation='linear'))  # Linear activation for regression task

In [ ]:
# Model summary
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Define early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)

In [ ]:
# Train the model
history = model.fit(
    X_train_scaled, y_train,
    epochs=150,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

## Model Evaluation

In [ ]:
# Plot training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

In [ ]:
# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse:.4f}')
print(f'Root Mean Squared Error: {rmse:.4f}')
print(f'R² Score: {r2:.4f}')

In [ ]:
# Plot actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual Final Grade')
plt.ylabel('Predicted Final Grade')
plt.title('Actual vs Predicted Final Grades')
plt.show()

## Feature Importance Analysis

In [ ]:
# Create a simple function to estimate feature importance
def get_feature_importance(model, X_scaled, feature_names):
    # Create a baseline prediction
    baseline_pred = model.predict(X_scaled)
    
    # Store importance scores
    importance = []
    
    # For each feature
    for i in range(X_scaled.shape[1]):
        # Create a copy of the data
        X_permuted = X_scaled.copy()
        
        # Shuffle the values of the current feature
        X_permuted[:, i] = np.random.permutation(X_permuted[:, i])
        
        # Predict with the permuted feature
        perm_pred = model.predict(X_permuted)
        
        # Calculate the increase in MSE
        mse_increase = mean_squared_error(baseline_pred, perm_pred)
        importance.append(mse_increase)
    
    # Create a DataFrame with feature names and importance scores
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importance
    })
    
    # Sort by importance
    feature_importance = feature_importance.sort_values('Importance', ascending=False)
    
    return feature_importance

# Get feature importance
feature_names = X.columns
feature_importance = get_feature_importance(model, X_test_scaled, feature_names)

In [ ]:
# Plot feature importance
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance)
plt.title('Feature Importance')
plt.tight_layout()
plt.show()

## Conclusion

In this notebook, we built an Artificial Neural Network model to predict student performance based on various factors. The model achieved an R² score of [value], indicating that it explains a significant portion of the variance in student performance.

The most important factors affecting student performance were found to be:
1. Previous academic performance
2. Study time
3. [Other important factors based on the feature importance analysis]

This model could be used by educational institutions to identify students who might need additional support to improve their academic performance.

## Future Work

1. Collect more data to improve model accuracy
2. Try different model architectures and hyperparameters
3. Include more features such as psychological factors, learning styles, etc.
4. Implement a more sophisticated feature selection process
5. Develop an early warning system for students at risk of poor performance